# Deploying ADK Agent on Vertex AI Agent Engine

## Overview

This notebook guides you through deploying and managing a conversational AI agent to **Vertex AI Agent Engine**. 

## Learning Goals

By the end of this notebook, you will understand how to:
* Set up required configuration and deploy ADK **Agents** to **Vertex AI Agent Engine**.
* Use the ADK **RemoteAgent** ans **RemoteSession** to execute agent interactions.
* Use the ADK to Manage (list/update) and delete agents that you have deployed to Vertex AI Agent Engine.

## ADK Agent deployment Options
Your ADK agent can be deployed to a range of different environments based on your needs for production readiness or custom flexibility:

#### Agent Engine in Vertex AI
Agent Engine is a fully managed auto-scaling service on Google Cloud specifically designed for deploying, managing, and scaling AI agents built with frameworks such as ADK.

[Learn more about deploying your agent to Vertex AI Agent Engine.](https://google.github.io/adk-docs/deploy/agent-engine/)

#### Cloud Run
Cloud Run is a managed auto-scaling compute platform on Google Cloud that enables you to run your agent as a container-based application.

[Learn more about deploying your agent to Cloud Run.](https://google.github.io/adk-docs/deploy/cloud-run/)

#### Google Kubernetes Engine (GKE)
Google Kubernetes Engine (GKE) is a managed Kubernetes service of Google Cloud that allows you to run your agent in a containerized environment. GKE is a good option if you need more control over the deployment as well as for running Open Models.

[Learn more about deploying your agent to GKE.](https://google.github.io/adk-docs/deploy/gke/)

### Agent Engine Overview
**Agent Engine** is a set of services that enables developers to deploy, manage, and scale AI Agents in production. It handles the infrastructure to scale agents in production so you can focus on creating applications. Agent Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Also its compatible with ADK, LangChain, LlamaIndex, or other Python frameworks.

Here are the services that Agent Engine offers, which you can use individually or in combination:

**Runtime:** Deploy and scale agents with a managed runtime and end-to-end management capabilities.

**Quality and evaluation**: Evaluate agent quality with the integrated Gen AI Evaluation service and optimize agents with Gemini model training runs.

**Example Store**: Store and dynamically retrieve few-shot examples to improve agent performance.

**Sessions**: Agent Engine Sessions lets you store individual interactions between users and agents, providing definitive sources for conversation context.

**Memory Bank**: Agent Engine Memory Bank lets you store and retrieve information from sessions to personalize agent interactions.

## Setup
This lab needs a special kernel to run, please run the following cell.
**NOTE: You can skip this step if you have already built the ADK Kernel from the previous Lab**

In [ ]:
!echo "Kernel installation started."
!cd ../../.. && make adk_kernel > /dev/null 2>&1
!echo "Kernel installation completed."

When it's completed, select the **`ADK Kernel`** on the top right before going forward in the notebook.<br>
It may take ~1 minutes until the kernel is shown after the installation.

## Install Packages

In [ ]:
import importlib
import os
import json
import warnings

import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

# Ignore all warnings
warnings.filterwarnings("ignore")

import logging

logging.basicConfig(level=logging.ERROR)

In [ ]:
LOCATION = "us-central1"
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"  # Use Vertex AI API

In [ ]:
%%bash
echo > adk_agents/.env "GOOGLE_CLOUD_LOCATION=$GOOGLE_CLOUD_LOCATION
GOOGLE_GENAI_USE_VERTEXAI=$GOOGLE_GENAI_USE_VERTEXAI
"

In [ ]:
MODEL = "gemini-2.0-flash"

## Prepare an Agent files

Here, let's reuse the basic agent files created in [building_agent_with_adk.ipynb](./building_agent_with_adk.ipynb) notebook.

If you haven't run the notebook, executed the cells below to create files.

In [ ]:
%%writefile ./adk_agents/agent1_weather_lookup/tools.py
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

In [ ]:
%%writefile ./adk_agents/agent1_weather_lookup/agent.py
from google.adk.agents import Agent
MODEL = "gemini-2.0-flash"

from .tools import get_weather

root_agent = Agent(
    name="weather_agent_v1",
    model=MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

In [ ]:
APP_NAME = "weather_info_app"
USER_ID = "user_1"

## Deploy to Vertex AI Agent Engine
Agent Engine is a fully managed Google Cloud service enabling developers to deploy, manage, and scale AI agents in production. 
Agent Engine handles the infrastructure to scale agents in production so you can focus on creating intelligent and impactful applications.

In [ ]:
LOCATION = "us-central1"
PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET_NAME = f"agent-deployment-{PROJECT}-bucket"
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Checking for the existence of BUCKET. Creating it if it doesn't exist:**

In [ ]:
!gsutil ls $BUCKET_URI || gsutil mb -l $LOCATION $BUCKET_URI

### Initialization

In [ ]:
vertexai.init(project=PROJECT, location=LOCATION, staging_bucket=BUCKET_URI)

### Prepare your agent for Agent Engine¶
Use `reasoning_engines.AdkApp()` to wrap your agent to make it deployable to Agent Engine

In [ ]:
from adk_agents.agent1_weather_lookup import agent

importlib.reload(agent)  # Force reload
adk_app = AdkApp(agent=agent.root_agent, enable_tracing=True)

### Deploy your agent to Agent Engine

Now you're ready to deploy your agent to Agent Engine in Vertex AI by calling `agent_engines.create()` along with:

1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.

This step may take several minutes to finish.

In [ ]:
DISPLAY_NAME = agent.root_agent.name

remote_app = agent_engines.create(
    adk_app,
    display_name=DISPLAY_NAME,
    description="Weather Agent v1",
    requirements=[
        "google-adk (>=1.5.0)",
        "google-genai (>=1.5.0,<2.0.0)",
    ],
    extra_packages=["./adk_agents/"],
)

print(f"Created remote agent: {remote_app.resource_name}")

Deployment takes a few minutes, during which the following steps happen in the background:

A bundle of the following artifacts are generated locally:
 - *.pkl a pickle file corresponding to local_agent.
 - requirements.txt a text file containing the package requirements.
 - dependencies.tar.gz a tar file containing any extra packages.

The bundle is uploaded to Cloud Storage (under the corresponding folder) for staging the artifacts.
The Cloud Storage URIs for the respective artifacts are specified in the PackageSpec.
The Vertex AI Agent Engine service receives the request and builds containers and starts HTTP servers on the backend.

Deployment latency depends on the total time it takes to install required packages.
Once deployed, remote_agent corresponds to an instance of agent that is running on Vertex AI and can be queried or deleted.
You can check deployed agents using Cloud Console: Vertex AI -> Agent Engine

[https://console.cloud.google.com/vertex-ai/agents/agent-engines]()

### Try ADK agent on Agent Engine
#### Create session (remote)
Just like you wouldn't start every text message from scratch, agents need context regarding the ongoing interaction.
Session is the ADK object designed specifically to track and manage these individual conversation threads.
When a user starts interacting with your agent, the SessionService creates a Session object to keep track for ongoing interactions for this user.
**where USER_ID is a user-defined ID with a character limit of 128.**

In [ ]:
remote_session = remote_app.create_session(user_id=USER_ID)

#### Lets explore the output for create_session (remote):
**id is the session ID, and app_name is the resource ID of the deployed agent on Agent Engine.**
A session represents an interaction between you and an AI agent. It tracks the conversation history, agent actions, and memory, allowing the agent to maintain context across interactions.

In [ ]:
print(f"ID (`id`):                {remote_session['id']}")
print(f"Application Name (`app_name`): {remote_session['appName']}")
print(f"User ID (`user_id`):         {remote_session['userId']}")
print(f"State (`state`):           {remote_session['state']}") # Note: Only shows initial state here
print(f"Events (`events`):         {remote_session['events']}") # Initially empty
print(f"Last Update (`last_update_time`): {remote_session['lastUpdateTime']:.2f}") # Unix timestamps in seconds

#### Send queries to your agent (remote)
Lets defines a helper function **format_adk_output** that processes the raw ADK Agent output.
It identifies each step: tool call, tool response, final text output and formats it clearly.

In [ ]:
def print_adk_output(event: dict):
    # Extract the first part of the content
    part = event.get('content', {}).get('parts', [{}])[0]
    author = event.get('author', 'unknown_agent')
    print(f"\n--- [Call from: {author}] ---")

    # Case 1: The model is calling a function/tool
    if 'function_call' in part:
        call = part['function_call']
        func_name = call.get('name', 'N/A')
        func_args = call.get('args', {})
        print("Tool Call:")
        print(f"  - Function: {func_name}")
        print(f"  - Arguments: {json.dumps(func_args, indent=2)}")

    # Case 2: The tool is returning a response
    elif 'function_response' in part:
        response = part['function_response']
        func_name = response.get('name', 'N/A')
        func_response = response.get('response', {})
        print("Tool Response:")
        print(f"  - From: {func_name}")
        # Pretty print the response JSON
        print(f"  - Data: {json.dumps(func_response, indent=2)}")

    # Case 3: The model is generating a final text response
    elif 'text' in part:
        text_response = part['text'].strip()
        print("Final Agent Response:")
        print(f"  -> {text_response}")

    else:
        print("Unknown step type.")

Now you can send a prompt to your remote agent using .stream_query to get the response:

In [ ]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="What's the weather in New York?",
):
    print_adk_output(event)

#### Inspect traces for an agent
A trace is a timeline of requests as your agent responds to each query.
Go to Trace Explorer in the Google Cloud console:
Select your Google Cloud project (corresponding to PROJECT_ID) at the top of the page.

The first row in the Gantt chart is for the trace. A trace is composed of individual spans, which represent a single unit of work, like a function call or an interaction with an LLM, with the first span representing the overall request. Each span provides details about a specific operation, such as the operation's name, start and end times, and any relevant attributes, within the request.
To learn more, see the [Cloud Trace documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/tracing).

#### List sessions (remote)
Vertex AI Agent Engine Sessions maintains the history of interactions between a user and agents.
Sessions provide definitive sources for long-term memory and conversation context.
In case of resuming existing conversations, you can get all existing sessions for a user:

In [ ]:
remote_app.list_sessions(user_id=USER_ID)

You can inspect existing sessions using Google Cloud console.
Go to Vertex AI in the Google Cloud console, and then navigate to the Agent Engine, open "Sessions" Tab for specific agent.

#### Get a specific session (remote)
Retrieving a specific Session (using its ID) so the agent can continue where it left off.
**To get a specific session, you need both the user ID and the session ID:**
While using your agent locally, session ID is stored in session.id, when using your agent remotely on Agent Engine, session ID is stored in remote_session["id"].

In [ ]:
remote_app.get_session(user_id=USER_ID, session_id=remote_session["id"])

### Manage deployed agents
Deployed agents are resources of type reasoningEngine in Vertex AI.

#### List all deployed agents for a given project and location:
In case if you need to list all active AI Agents deployments you cau use `agent_engines.list()` method.
Each deployed agent has a unique identifier ***Name*** and fully qualified resource name ***Resource Name***

In [ ]:
agents_list = agent_engines.list()
for remote_agent in agents_list:
    print("\n------")
    print(f"Display Name: {remote_agent.display_name}")
    print(f"Name: {remote_agent.name}")
    print(f"Resource Name: {remote_agent.resource_name}")
    print(f"Create Time: {remote_agent.create_time}")

#### Filter results by display_name
In case if you need to get specific agent by only display_name:

In [ ]:
DISPLAY_NAME = "weather_agent_v1"

In [ ]:
agents_list = agent_engines.list(filter=f'display_name="{DISPLAY_NAME}"')
if agents_list:
    AGENT_RESOURCE_ID = remote_agent.name
    print(f"Agent Name: {remote_agent.name}")
    AGENT_RESOURCE_NAME = remote_agent.resource_name
    print(f"Resource Name: {remote_agent.resource_name}")
else:
    print(f'Cant find deployed agent with the display_name="{DISPLAY_NAME}"')

#### Get a deployed agent
The following code lets you get a specific deployed agent:

In [ ]:
remote_agent = agent_engines.get(AGENT_RESOURCE_NAME)
remote_agent

Alternately, you can provide the fully qualified resource name:

In [ ]:
remote_agent = agent_engines.get(AGENT_RESOURCE_ID)
remote_agent

### Update a deployed agent
A very common use case is to update a deployed agent with a new version that might contain new capabilities or bug fixes.

#### Prepare an updated Agent files
Here, let's reuse the basic agent files created in building_agent_with_adk.ipynb notebook.

If you haven't run the notebook, executed the cells below to create files.

In [ ]:
!cp ./adk_agents/agent1_weather_lookup/tools.py ./adk_agents/agent2_sub_agent/

In [ ]:
%%writefile -a ./adk_agents/agent2_sub_agent/tools.py

def say_hello(name: str = "there") -> str:
    """Provides a simple greeting, optionally addressing the user by name.

    Args:
        name (str, optional): The name of the person to greet. Defaults to "there".

    Returns:
        str: A friendly greeting message.
    """
    if name is None or name.strip() == "":
        name = "there"
    print(f"--- Tool: say_hello called with name: {name} ---")
    return f"Hello, {name}!"

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    print(f"--- Tool: say_goodbye called ---")
    return "Goodbye! Have a great day."

In [ ]:
%%writefile ./adk_agents/agent2_sub_agent/agent.py
from google.adk.agents import Agent
MODEL = "gemini-2.0-flash"

from .tools import get_weather, say_hello, say_goodbye

# --- Greeting Agent ---
greeting_agent = Agent(
    model=MODEL,
    name="greeting_agent",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                "Use the 'say_hello' tool to generate the greeting. "
                "If the user provides their name, make sure to pass it to the tool. "
                "Do not engage in any other conversation or tasks.",
    description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
    tools=[say_hello],
)

In [ ]:
%%writefile -a ./adk_agents/agent2_sub_agent/agent.py

# --- Farewell Agent ---
farewell_agent = Agent(
    model=MODEL,
    name="farewell_agent",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                "Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
    tools=[say_goodbye],
)

In [ ]:
%%writefile -a ./adk_agents/agent2_sub_agent/agent.py

root_agent = Agent(
    name="weather_agent_v2", # Give it a new version name
    model=MODEL,
    description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
    instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                "You have specialized sub-agents: "
                "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                "If it's a weather request, handle it yourself using 'get_weather'. "
                "For anything else, respond appropriately or state you cannot handle it.",
    tools=[get_weather],
    sub_agents=[greeting_agent, farewell_agent]
)

#### Update a deployed agent to an updated agent
You can update one or more fields of the deployed agent at the same time, but you have to specify at least one of the fields to be updated. The amount of time it takes to update the deployed agent depends on the update being performed, but it generally takes between a few seconds to a few minutes.

To update a deployed agent (***weather_agent_v1***, corresponding to AGENT_RESOURCE_ID) to an updated agent (***weather_agent_v2*** corresponding to UPDATED_AGENT_APP):

In [ ]:
from adk_agents.agent2_sub_agent import agent as agent2

importlib.reload(agent2)  # Force reload

UPDATED_AGENT_APP = AdkApp(agent=agent2.root_agent, enable_tracing=True)

agent_engines.update(
    resource_name=AGENT_RESOURCE_ID,
    agent_engine=UPDATED_AGENT_APP,
    description="Weather Agent v2",
    display_name=agent2.root_agent.name,
    requirements=[
        "google-adk (>=1.5.0)",
        "google-genai (>=1.5.0,<2.0.0)",
    ],
    extra_packages=["./adk_agents/"],
)

Checking changed result, after agent update:

In [ ]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="Hello!",
):
    for key, label in event.items():
        print("\n------>")
        print(f"{key} : {label}")

In [ ]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="Bye!",
):
    for key, label in event.items():
        print("\n------>")
        print(f"{key} : {label}")

### Clean up
After you have finished, it is a good practice to clean up your cloud resources. 
You can delete the deployed Agent Engine instance by using the next code:

In [ ]:
# remote_app.delete(force=True)

Alternatively, you can call agent_engines.delete() to delete the deployed agent corresponding to AGENT_RESOURCE_ID in the following way:


In [ ]:
# agent_engines.delete(AGENT_RESOURCE_ID)

Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.